# MotoGP Regression Part 2 - Combining Separate Data Files  
Ankur Vishwakarma  
Metis Winter 2018

<img src="http://css.motogp.com/w2015/img/logos/motogp.svg?version=a4b2faf82fe5880934375619ed3dac5aadb10ea007af6f4f770b682e0d16dbf6" width="120" align="left"/>

In this notebook, all the separate CSVs from scraped sites are combined. Each year's results data is loaded and then cleaned. Additional data is brought in from the Racetrack_data.csv file. Lastly, all dataframes are combined into one large dataframe with all the data.

In [1]:
# import necessary modules
import time
from pprint import pprint
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import calendar

In [2]:
# load racetrack data that'll be needed for all the results files
rc = pd.read_csv('Racetrack_data.csv', index_col=0)

In [3]:
rc.head()

,GP,track_length_km,l_corners,r_corners,width_m,straight_m,GP_avg_speed,gp_dist,m2_dist,m3_dist
0,GBR - Silverstone Circuit,5.9,8,10,15.0,770.0,173.7,118.0,106.2,100.3
1,NED - TT Circuit Assen,4.5,6,12,14.0,487.0,169.9,118.1,109.0,99.9
2,AUS - Phillip Island,4.4,7,5,13.0,900.0,176.4,120.1,111.2,102.3
3,ITA - Autodromo del Mugello,5.2,6,9,14.0,1141.0,174.2,120.6,110.1,104.9
4,AME - Circuit Of The Americas,5.5,11,9,15.0,1200.0,157.9,115.8,104.7,99.2


In [7]:
# define the years we want to combine data for
years = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

# function to parse dates as we read in results csv files
def dateparser(dstr):
    """ Returns a datetime object for any date string in the format
        Month, dd, yyyy """
    d = dict((v,k) for k,v in enumerate(calendar.month_name))
    if type(dstr) != float:
        mon, dd, yyyy = dstr.split(',')
        date_str = '/'.join([str(d[mon]), dd, yyyy])
        return pd.to_datetime(date_str, format='%m/%d/%Y')
    else:
        pass
    return None

# function that'll help determine which string values are actually numbers
def isNumber(x):
    if pd.isnull(x):
        return False
    try:
        float(x)
        return True
    except ValueError:
        pass
    return False

def subset(string, chars):
    if pd.isnull(string):
        return None
    else:
        return string[:chars]

# function that'll convert finishing time into a proper timedelta value
def timeparser(time_series):
    
    new_time = []
    for index, item in enumerate(time_series):
        prev_case = True
        if pd.isnull(item):
            new_time.append(None)
            continue
        
        if 'Lap' in item:
            new_time.append(None)
            prev_case = False
            
        if ('+' in item) & ("'" not in item):
            t = datetime.strptime(item, "+%S.%f")
            t_delta = timedelta(hours=t.hour, minutes=t.minute,seconds=t.second, microseconds=t.microsecond)
            t_delta = t_delta + base_delta
            new_time.append(t_delta)
            prev_case = False
            
        if ('+' in item) & ("'" in item):
            t = datetime.strptime(item, "+%M'%S.%f")
            t_delta = timedelta(hours=t.hour, minutes=t.minute,seconds=t.second, microseconds=t.microsecond)
            t_delta = t_delta + base_delta
            new_time.append(t_delta)
            prev_case = False
            
        if prev_case:
            base_time = datetime.strptime(item, "%M'%S.%f")
            base_delta = timedelta(hours=base_time.hour, minutes=base_time.minute,
                                   seconds=base_time.second, microseconds=base_time.microsecond)
            new_time.append(base_delta)
            
    return new_time

In [8]:
dfs = []
rows_read = []
for yr in reversed(years):
    print(yr, end=', ')
    df = pd.read_csv('/Archive/'+yr+'_data.csv', index_col=0, parse_dates=['Date'], date_parser=dateparser)
    df['Track_Temp'] = df['Track_Temp'].map(lambda x: int(x[:2]) if isNumber(subset(x,2)) else x)
    df['Air_Temp'] = df['Air_Temp'].map(lambda x: int(x[:2]) if isNumber(subset(x,2)) else x)
    df['Humidity'] = df['Humidity'].map(lambda x: float(x[:2])/100 if isNumber(subset(x,2)) else x)
    df['Finish_Time'] = timeparser(df.Time)
    df['GP'] = df.TRK.map(lambda x: x+' - ') + df.Track.map(lambda x: x.split(' - ')[1])
    df = df.merge(rc, on='GP', how='left')
    rows_read.append(len(df))
    dfs.append(df)
print('Complete!')

2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, Complete!


In [9]:
# save to master CSV
result = pd.concat(dfs, ignore_index=True)
fn = 'MotoGP_2005_2017.csv'
result.to_csv(fn)